### Dynamic Language

#### First Order
함수도 데이터(객체)에 해당, 복사가능


In [1]:
def hello(x):
    print('Hi i am a function')
    
firs = hello
print(firs(12)) # None?? hello의 리턴값이 없으므로 일종의 side effect -> 항상 리턴값 만들어라

Hi i am a function
None


In [2]:
firs(12)

Hi i am a function


In [3]:
Ankit = print
Ankit("Hi I'm print")

Hi I'm print


In [4]:
del hello # 함수정의를 삭제해도 복사된 firs 객체는 메모리에 존재
print(firs(12))

Hi i am a function
None


In [6]:
def hello(x):
    print('Hi i am a function')
    return 1

In [9]:
firs = hello(1) # 함수가 아니라 리턴값을 받아오면
print(firs(12))

Hi i am a function


TypeError: 'int' object is not callable

In [13]:
callable(firs) # callable 하지않음

False

In [14]:
def hello(x):
    print('Hi i am a function')
    return hello # 함수의 리턴값으로 함수인 자기자신

In [16]:
firs = hello(1)

Hi i am a function


In [17]:
print(firs(12))

Hi i am a function
<function hello at 0x00000224F0FCE2F0>


In [19]:
hello(1)(2)(3)

Hi i am a function
Hi i am a function
Hi i am a function


<function __main__.hello>

In [23]:
def bye():
    print('look')
    
def hello(fun):
    print('hello')
    print('This is my real fun', fun.__name__)
    fun()

In [24]:
hello(bye)

hello
This is my real fun bye
look


In [25]:
print(hello(bye))

hello
This is my real fun bye
look
None


함수 콜해서 리스트의 원소로 넣기도 가능

이 과정에서 return이 미뤄지는 일종의 lazy evaluation

-> 디버깅시 print 쓰는게 한계있다

In [26]:
def f1():
    print('look')
    return 0
def f2():
    print('hello')
    return 0
def f3(r):
    print('oh', r)
    return 0

fs = [f1(),f2(),f3(12)]

for f in fs:
    print(f)

look
hello
oh 12
0
0
0


### closure:

__내부함수를 반환하는 외부함수__ 구조

->동적 함수생성을 가능하게 한다

In [35]:
def hello(x):
    x=12
    def bye():
        print(x)
        return 'This is bye\'s return'
    return bye 

er = hello(13)
print(er())

12
This is bye's return


In [37]:
def hello(x):
    def by():
        print(x)
        return x.upper()
    return by

print(hello('ankit')())

ankit
ANKIT


In [39]:
def funcwrapper(y):
    def addone(x):
        return x + y +1
    return addone

print(callable(funcwrapper(1))) # funcwrapper(1)이라는 함수가 생성된다
funcwrapper(1)(2)

True


4

데코레이터가 붙은 함수의 인자로 밑에 정의된 함수를 넣는다

In [40]:
def hello(func):
    def bye(x):
        print('before dec')
        func(x)
        print('after dec')
    return bye

@hello
def hi(sa):
    print('hk', sa)
    
hi(1)

before dec
hk 1
after dec


In [ ]:
def hello(func):
    def wrapper(x):
        print("This is stuff before dec")
        func(x)
        print("This is stuff after dec")
    return wrapper

@hello


3층구조 데코레이터 -> 데코레이터의 인자를 받는다( @app.route('/') )

In [58]:
def extrafun(hi):
    def decorator(func):
        def wrapper(x):
            print("This is stuff before dec")
            func(x)
            print("This is stuff after dec")
        return wrapper
    return decorator

@extrafun(1)
def newfunc(c):
    print("this is new function", 'passed par is:',c)

newfunc(30)

This is stuff before dec
this is new function passed par is: 30
This is stuff after dec
